In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
# standard numerical library imports
import numpy as np

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.datasets import qg_jets
from energyflow.utils import data_split, remap_pids, to_categorical

Using TensorFlow backend.
/home/spark/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/spark/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/spark/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/spark/miniconda3/

In [3]:
# attempt to import sklearn
try:
    from sklearn.metrics import roc_auc_score, roc_curve
except:
    print('please install scikit-learn in order to make ROC curves')
    roc_curve = False

In [4]:

# attempt to import matplotlib
try:
    import matplotlib.pyplot as plt
except:
    print('please install matploltib in order to make plots')
    plt = False

In [5]:
# data controls, can go up to 2000000 for full dataset
train, val, test = 85000, 10000, 15000
# train, val, test = 1000000, 200000, 200000
use_pids = False

# network architecture parameters
Phi_sizes, F_sizes = (100, 100, 128), (100, 100, 100)
# Phi_sizes, F_sizes = (100, 100, 256), (100, 100, 100)

# network training parameters
num_epoch = 5
batch_size = 500


In [6]:
import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
import torch.utils.data as utils
import torch.nn.init as init
from torch.autograd import Variable

In [7]:
f = pd.read_hdf("/data/t3home000/spark/LHCOlympics/data/events_anomalydetection.h5")

In [8]:
f.head

<bound method NDFrame.head of              0         1         2          3         4         5     \
0        0.324101 -0.361158  2.737669   0.409859 -2.429939  0.729830   
1        0.646304 -0.539460 -1.386258   0.471293 -1.636572  0.751657   
2        0.325172 -0.833948  2.404844   1.295058 -2.089618 -1.873342   
3        0.290918 -2.200063  1.630132   0.565028 -1.714345 -2.617103   
4        0.526330 -2.349110 -1.745532   0.542491 -2.080352 -3.044045   
...           ...       ...       ...        ...       ...       ...   
1099995  0.372368 -1.283898  2.680089   0.511526 -0.216247  1.011824   
1099996  0.587996 -2.347485  0.104980   0.325682 -1.943584 -2.330028   
1099997  0.631419 -0.597308 -2.114541   3.165008 -0.968924 -0.729551   
1099998  5.084272 -1.004487  1.772670   4.767743 -0.878143  1.713857   
1099999  0.402804 -1.502353 -0.956501  39.444363 -1.502638 -2.056680   

              6         7         8           9     ...  2091  2092  2093  \
0         0.867922 -2.267777

In [9]:
dt = f.values

In [10]:
dt.shape

(1100000, 2101)

In [11]:
dt2 = dt[:,:2100]

In [12]:
dt3 = dt[:,-1]

In [13]:
dt3.shape

(1100000,)

In [14]:
dt3 = dt3.reshape(1,len(dt3))

In [15]:
dt3.shape

(1, 1100000)

In [16]:
dt2 = dt2.reshape((len(dt2), len(dt2[0])//3, 3))

In [17]:
dt2.shape

(1100000, 700, 3)

In [18]:
# data controls
num_data = 1100000
test_frac = 0.2

# efp parameters
dmax = 10
measure = 'hadr'
beta = 0.5


In [19]:
print('Calculating d <= {} EFPs for {} jets... '.format(dmax, num_data), end='')

Calculating d <= 10 EFPs for 1100000 jets... 

In [20]:
efpset = ef.EFPSet(('d<=', dmax), measure='hadr', beta=beta)

In [21]:
masked_X = [x[x[:,0] > 0] for x in dt2]

In [23]:
test = dt2[:100]

In [25]:
test.shape

(100, 700, 3)

In [27]:
test_X = [x[x[:,0] > 0] for x in test]

AttributeError: 'list' object has no attribute 'shape'

In [30]:
X = efpset.batch_compute(test_X)

Process ForkPoolWorker-28:
Traceback (most recent call last):
  File "/home/spark/miniconda3/envs/myenv/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/spark/miniconda3/envs/myenv/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/spark/miniconda3/envs/myenv/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/spark/miniconda3/envs/myenv/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/spark/miniconda3/envs/myenv/lib/python3.6/site-packages/energyflow/base.py", line 147, in _batch_compute_func
    return self.compute(event, batch_call=True)
  File "/home/spark/miniconda3/envs/myenv/lib/python3.6/site-packages/energyflow/efp.py", line 565, in compute
    results = [efp._efp_compute(zs, thetas_dict) for efp in self._efps]
  File "/home/spark/miniconda3/envs/mye

KeyboardInterrupt: 